## Module 17 Challenge, Deliverable 3:   Ensemble Classifiers to Predict Credit Risk, <br>Balanced Random Forest Classifier and Easy Ensemble AdaBoost classifier

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets

## Read the CSV and Perform Basic Data Cleaning

In [7]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [8]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [9]:
# Create our features. 
#X = pd.get_dummies(df, columns=["home_ownership", "verification_status", "issue_d", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type", "hardship_flag", "debt_settlement_flag"])
X = pd.get_dummies(df.drop('loan_status', axis=1))

# Create our target
y = df['loan_status'].tolist()

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
# Check the balance of our target values
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

## Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier. BalancedRandomForestClassifier(random_state=1)
rf_model = RandomForestClassifier(n_estimators=500, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [51]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test)

In [52]:
# Calculated the balanced accuracy score  0.7885466545953005
balanced_accuracy_score(y_test, predictions)

0.6830221521918328

In [53]:
# Display the confusion matrix.  array([[   71,    30],
#                                       [ 2153, 14951]])
cm = confusion_matrix(y_test, predictions)
cm

array([[   37,    64],
       [    5, 17099]], dtype=int64)

In [54]:
# Create a DataFrame from the confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


In [55]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

   high_risk       0.88      0.37      0.52       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.94      0.68      0.76     17205
weighted avg       1.00      1.00      1.00     17205



In [56]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.88      0.37      1.00      0.52      0.61      0.34       101
   low_risk       1.00      1.00      0.37      1.00      0.61      0.39     17104

avg / total       1.00      1.00      0.37      1.00      0.61      0.39     17205



In [13]:
# Print the imbalanced classification report

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.62     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.62     17205



In [58]:
print("Balanced Random Forest Classifier")
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, predictions)}")

print(f"Confusion Matrix: ")
display(cm_df)

print("Imbalanced Classification Report: ")
print(classification_report_imbalanced(y_test, predictions))

Balanced Random Forest Classifier
Balanced Accuracy Score: 0.6830221521918328
Confusion Matrix: 


,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


Imbalanced Classification Report: 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.88      0.37      1.00      0.52      0.61      0.34       101
   low_risk       1.00      1.00      0.37      1.00      0.61      0.39     17104

avg / total       1.00      1.00      0.37      1.00      0.61      0.39     17205



In [22]:
# Calculate feature importance in the Random Forest model.
importances = [rf_model.feature_importances_]
importances

[array([0.01518763, 0.01251118, 0.02147159, 0.01231995, 0.01745217,
        0.00400446, 0.00521694, 0.00835228, 0.00278403, 0.01464125,
        0.01160755, 0.01926514, 0.02111947, 0.05475104, 0.06686221,
        0.0657326 , 0.07486923, 0.01211519, 0.        , 0.        ,
        0.06380464, 0.00171385, 0.        , 0.        , 0.00518971,
        0.01349905, 0.00625797, 0.00637361, 0.00419482, 0.0074497 ,
        0.0122846 , 0.01384658, 0.01117064, 0.0056273 , 0.0075976 ,
        0.0157521 , 0.01164238, 0.01233685, 0.0069534 , 0.0085127 ,
        0.00845784, 0.00979712, 0.01466131, 0.01441276, 0.0137198 ,
        0.00034778, 0.        , 0.01581678, 0.01573585, 0.00941275,
        0.00863488, 0.00655064, 0.01106567, 0.01037827, 0.00353701,
        0.00695829, 0.00770071, 0.00712845, 0.00853204, 0.01052011,
        0.00807636, 0.01023909, 0.00744191, 0.00836243, 0.        ,
        0.        , 0.00164462, 0.00693255, 0.00859778, 0.00650776,
        0.00259742, 0.        , 0.01403945, 0.01

In [23]:
# List the features sorted in descending order by feature importance.
#sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances = pd.DataFrame(rf_model.feature_importances_, index=X_train.columns, columns=['Importances']).sort_values('Importances', ascending=False)
importances.head(15)

,Importances
total_rec_int,0.074869
total_pymnt_inv,0.066862
total_rec_prncp,0.065733
last_pymnt_amnt,0.063805
total_pymnt,0.054751
installment,0.021472
out_prncp_inv,0.021119
out_prncp,0.019265
dti,0.017452
mo_sin_old_il_acct,0.015817


## Easy Ensemble AdaBoost Classifier

In [24]:
# Train the EasyEnsembleClassifier.  EasyEnsembleClassifier(n_estimators=100, random_state=1)
# YOUR CODE HERE
adaboost = AdaBoostClassifier(n_estimators=1000, learning_rate=1,random_state=1)
model = adaboost.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [36]:
# Calculated the balanced accuracy score.  0.9316600714093861
balanced_accuracy_score(y_test, y_pred)

0.7325563356148524

In [28]:
# Calculate the accuracy score - for Reference. 
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.9966288869514676

In [30]:
# Display the confusion matrix.  array([[   93,     8],
#                                       [  983, 16121]])
# YOUR CODE HERE
cm = confusion_matrix(y_test, y_pred)
cm

array([[   47,    54],
       [    4, 17100]], dtype=int64)

In [32]:
# Create a DataFrame from the confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
cm_df

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,47,54
Actual 1,4,17100


In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.92      0.47      0.62       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.96      0.73      0.81     17205
weighted avg       1.00      1.00      1.00     17205



In [35]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.92      0.47      1.00      0.62      0.68      0.44       101
   low_risk       1.00      1.00      0.47      1.00      0.68      0.49     17104

avg / total       1.00      1.00      0.47      1.00      0.68      0.49     17205



In [18]:
# Print the imbalanced classification report

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



In [50]:
print("Easy Ensemble AdaBoost Classifier")
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, y_pred)}")

print(f"Confusion Matrix: ")
display(cm_df)

print("Imbalanced Classification Report: ")
print(classification_report_imbalanced(y_test, y_pred))

Easy Ensemble AdaBoost Classifier
Balanced Accuracy Score: 0.7325563356148524
Confusion Matrix: 


,Predicted 0,Predicted 1
Actual 0,47,54
Actual 1,4,17100


Imbalanced Classification Report: 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.92      0.47      1.00      0.62      0.68      0.44       101
   low_risk       1.00      1.00      0.47      1.00      0.68      0.49     17104

avg / total       1.00      1.00      0.47      1.00      0.68      0.49     17205

